In [1]:
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys
sys.path.insert(0, "../")
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.models import MobileNetV2

from models import ModelWrapper_TorchVisionNet
from dataset import VoiceAntiSpoofDataset
from utils import read_fromBaseline, read_scipy
from Metrics import compute_err
#import DftSpectrogram
from librosa.feature import mfcc
import glob


from kekas import Keker, DataOwner
from kekas.callbacks import Callback, ProgressBarCallback
from kekas.metrics import accuracy



/home/vfilin/anaconda3/envs/idrnd/lib/python3.6/site-packages/kekas/keker.py:9: UserWarning: Error 'No module named 'apex''' during importing apex library. To use mixed precison you should install it from https://github.com/NVIDIA/apex
  warnings.warn(f"Error '{e}'' during importing apex library. To use mixed precison"


In [2]:
MN2 = MobileNetV2()
MN2.classifier[1] = nn.Linear(1280, 2, bias=False)
model = ModelWrapper_TorchVisionNet(MN2).to('cuda')
sum(p.numel() for p in model.parameters() if p.requires_grad)

2226438

In [9]:
dataset_dir = '/home/vfilin/Desktop/voice_antispoof_contest/Training_Data/'
print("Num samples:", len(glob.glob(os.path.join(dataset_dir, '**/*.wav'), recursive=True)))
dataset = VoiceAntiSpoofDataset(dataset_dir, 'all', read_scipy, transform=[lambda x: mfcc(x),  
                                lambda x: x[None, ...].astype(np.float32)])
dataset_val_dir = '/home/vfilin/Desktop/voice_antispoof_contest/validationASV/'
dataset_val = VoiceAntiSpoofDataset(dataset_val_dir, 'all', read_scipy, transform=[lambda x: mfcc(x),  
                                lambda x: x[None, ...].astype(np.float32)])
batch_size = 24
num_workers = 12
#dataset.data = dataset.data[:48] 
#dataset.labels = dataset.labels[:48]
#dataset_val.data = dataset_val.data[:48] + dataset_val.data[-24:]
#dataset_val.labels = dataset_val.labels[:48] + dataset_val.labels[-24:]
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)
val_dl = DataLoader(dataset_val, batch_size=batch_size, num_workers=num_workers, shuffle=False)

Num samples: 50000


In [10]:
dataowner = DataOwner(dataloader, val_dl, None)
weights = torch.tensor([5, 1.25])
weights = weights / weights.sum()
weights = weights.to('cuda')
criterion = nn.CrossEntropyLoss(weight=weights)

In [11]:
def step_fn(model: torch.nn.Module,
            batch: torch.Tensor) -> torch.Tensor:
    """Determine what your model will do with your data.

    Args:
        model: the pytorch module to pass input in
        batch: the batch of data from the DataLoader

    Returns:
        The models forward pass results
    """
    
    voice = batch['data']
    return model(voice)
class ScoreCallback(Callback):
    def __init__(self, preds_key, target_key, metric_fn):

        self.preds_key = preds_key
        self.target_key = target_key
        self.human_probs_for_human = []
        self.human_probs_for_spoof = []
        self.metric_fn = metric_fn
        
    def on_batch_end(self, i, state) -> None:
        if state.core.mode == "val":
            preds = state.core.out[self.preds_key].cpu().numpy()
            targets = state.core.batch[self.target_key].cpu().numpy()
            tmp = preds[targets == 0, 0]
            if len(tmp) > 0:
                self.human_probs_for_human.append(tmp)
            tmp = preds[targets == 1, 0]
            if len(tmp) > 0:
                self.human_probs_for_spoof.append(tmp)
    
    def on_epoch_end(self, epoch, state) -> None:
        if state.core.mode == "val":
            print("_"*50)
            print("Metric computation")
            self.human_probs_for_human = np.hstack(self.human_probs_for_human)
            self.human_probs_for_spoof = np.hstack(self.human_probs_for_spoof)
            eer, threshold = self.metric_fn(self.human_probs_for_human, self.human_probs_for_spoof)
            self.human_probs_for_human = []
            self.human_probs_for_spoof = []
            print('score 1 (eer): {0:.7f} threshold (?): {1:.4f}'.format(
                eer, threshold))


In [12]:
keker = Keker(model=model,
              dataowner=dataowner,
              criterion=criterion,
              step_fn=step_fn,                    # previosly defined step function
              target_key="label",                 # remember, we defined it in the reader_fn for DataKek? 
              opt=torch.optim.Adam,               # optimizer class. if note specifiyng, 
                                                  # an SGD is using by default
              opt_params={"weight_decay": 1e-5},
              callbacks=[ScoreCallback('preds', 'label', compute_err)],
                 metrics={"acc": accuracy})

In [ ]:
keker.kek(lr=1e-3,
          epochs=25,
          sched=torch.optim.lr_scheduler.MultiStepLR,       # pytorch lr scheduler class
          sched_params={"milestones":torch.tensor([12, 20]), "gamma": 0.1,},
         cp_saver_params={
              "savedir": "./checkpoints",
         "metric":"acc"})  # schedulres kwargas in dict format

Epoch 1/25:  76% 1919/2514 [07:29<02:26,  4.06it/s, loss=0.6778]